# 3: Income Effect only on Non-Car

Model 3 posits the effect of income on all the automobile modes (drive alone, shared ride 2, and shared ride 3+) is the same, but the effect is different for the other modes. We include this constraint by setting the income coefficients in the utilities of the automobile modes to be equal. In this case, we set them to zero since drive alone is the reference mode. (pp. 108)

We set the terms to zero can be done by omitting the terms altogether.

Import larch and verify that the version number is correct (as of 1/30/2024 should be 6.0.17)

In [ ]:
import larch

In [ ]:
larch.__version__

This example is a mode choice model built using the MTC example dataset. First we create the DB and Model objects:

In [ ]:
d = larch.examples.MTC(format="dataset")
d

In [ ]:
m = larch.Model(d, compute_engine="numba")

Then we can build up the utility function. We’ll use some :ref:idco data first, using the Model.utility.co attribute. This attribute is a dict-like object, to which we can assign :class:LinearFunction objects for each alternative code.

In [ ]:
from larch import PX, P, X

m.utility_co[2] = P("ASC_SR2")
m.utility_co[3] = P("ASC_SR3P")
m.utility_co[4] = P("ASC_TRAN") + P("hhinc#4") * X("hhinc")
m.utility_co[5] = P("ASC_BIKE") + P("hhinc#5") * X("hhinc")
m.utility_co[6] = P("ASC_WALK") + P("hhinc#6") * X("hhinc")

Next we’ll use some idca data, with the utility_ca attribute. This attribute is only a single :class:LinearFunction that is applied across all alternatives using :ref:idca data. Because the data is structured to vary across alternatives, the parameters (and thus the structure of the :class:LinearFunction) does not need to vary across alternatives.

In [ ]:
m.utility_ca = PX("tottime") + PX("totcost")

Lastly, we need to identify idca Format data that gives the availability for each alternative, as well as the number of times each alternative is chosen. (In traditional discrete choice analysis, this is often 0 or 1, but it need not be binary, or even integral.)


In [ ]:
m.availability_ca_var = "avail"
m.choice_ca_var = "chose"

And let’s give our model a descriptive title.

In [ ]:
m.title = "MTC Example 3, Zeroed Shared"

We can view a summary of the choices and alternative availabilities to make sure the model is set up correctly.

In [ ]:
m.choice_avail_summary()

We’ll set a parameter cap (bound) at +/- 20, which helps improve the numerical stability of the optimization algorithm used in estimation.

In [ ]:
m.set_cap(20)

Having created this model, we can then estimate it:

In [ ]:
assert m.compute_engine == "numba"

In [ ]:
result = m.maximize_loglike(stderr=True)
m.calculate_parameter_covariance()
m.loglike()

In [ ]:
m.parameter_summary()

It is a little tough to read this report because the parameters show up in alphabetical order. We can use the reorder method to fix this and group them systematically:

In [ ]:
m.ordering = (
    (
        "LOS",
        "totcost",
        "tottime",
    ),
    (
        "ASCs",
        "ASC.*",
    ),
    (
        "Income",
        "hhinc.*",
    ),
)

In [ ]:
m.parameter_summary()

Finally, let's print model statistics.  Note that if you want LL at constants you need to run a separate model.

In [ ]:
m.estimation_statistics()

In [ ]:
# TEST
revealed_x = dict(zip(m.pnames, result.x))

In [ ]:
# TEST
from pytest import approx

expected_x = {
    "ASC_BIKE": -2.3989187974851647,
    "ASC_SR2": -2.3041966926745103,
    "ASC_SR3P": -3.7034452031908507,
    "ASC_TRAN": -0.6966955353312382,
    "ASC_WALK": -0.2269314175275228,
    "hhinc#4": -0.00486415050538345,
    "hhinc#5": -0.012431555290989793,
    "hhinc#6": -0.009365241345664176,
    "totcost": -0.004912235245649822,
    "tottime": -0.05135135046958243,
}
for k in expected_x:
    assert revealed_x[k] == approx(expected_x[k], 2e-2), (
        f"{k}, {revealed_x[k] / expected_x[k]}"
    )